### IMPORTS

In [1]:
import pandas as pd
import numpy as np
import pandas_gbq
import datetime
from dateutil.relativedelta import relativedelta
from Roster import *
from gspread_pandas import Spread, conf

### CREDENCIALES

In [2]:
cred = conf.get_config('C:\\Users\\micaela.fuchs\\Anaconda', 'PedidosYa-6e661fd93faf.json')

### CONSTANTES

In [3]:
# Fechas
today = datetime.date.today()
ilm = str((today - relativedelta(months=1)).replace(day=1))
flm = str(today - relativedelta(days=today.day))

### GOOGLE SHEETS

In [4]:
# Partners PEYA
sheet_id = '1HmAvHYbJJa3JyRTgRJGKXataQB_TvFwNzyo8a6qXd2o'
wks_name = 'Partners PEYA'
sheet = Spread(sheet_id, wks_name, config=cred)
partners = sheet.sheet_to_df(index=0,header_rows=1)

In [5]:
# Sales Growth
sheet_id = '1JNywQTVzEQKRwqrJRkpzjiXx5Ly-FldtBMfeSYHuL7w'
wks_name = 'Sales Growth'
sheet = Spread(sheet_id, wks_name, config=cred)
sl_growth = sheet.sheet_to_df(index=0,header_rows=1)

In [6]:
# Hago lista de Sales Growth
sl_growth = sl_growth['Sales'].to_list()
# Paso todo a mayusculas
sl_growth = [x.upper() for x in sl_growth]

### QUERIES

In [7]:
# 57MB
q_cw = '''SELECT p.salesforce_id AS Grid,
       sfo.Business_Type AS Tipo,
       sfc.Name__c AS Nombre,
       cw.account_owner AS Owner,
       DATE(cw.closed_date) AS Fecha_Cierre,
       MAX(sfc.Commission) AS Comision,
FROM `peya-bi-tools-pro.il_core.fact_sales_executives_closed_won_v2` AS cw
LEFT JOIN `peya-bi-tools-pro.il_core.dim_partner` AS p ON cw.grid = p.salesforce_id
LEFT JOIN `peya-data-origins-pro.cl_salesforce.opportunity` AS sfo ON cw.oportunity_unique_id = sfo.Id
LEFT JOIN `peya-data-origins-pro.cl_salesforce.contract` AS sfc ON sfo.Id = sfc.Id_Opportunity
WHERE DATE(cw.closed_date) BETWEEN DATE('{0}') AND DATE('{1}')
      AND p.country_id = 3
      AND sfo.Business_Type IN ('New Business','Win Back','Franchise Extension','New Bussiness')
GROUP BY 1,2,3,4,5'''.format(ilm,flm)

In [8]:
# Descargo la data
hue_cw = pd.io.gbq.read_gbq(q_cw, project_id='peya-argentina', dialect='standard')

Downloading: 100%|██████████| 2682/2682 [00:01<00:00, 1776.05rows/s]


In [9]:
# Copio las bases
cw = hue_cw.copy()

### TRABAJO

In [10]:
# Mando el Propietario a Mayuscula
cw['Owner'] = cw['Owner'].apply(lambda x: x.upper())
# Me quedo con las CW de Growth
growth_cw = cw[cw['Owner'].isin(sl_growth)].copy()
# Cambio las fechas a Str
growth_cw['Fecha_Cierre'] = growth_cw['Fecha_Cierre'].astype(str)

In [11]:
# Selecciono las columnas necesarias
cols = ['Id','Grid','Name','City','Area','Business']
# Doy los formatos necesarios
partners['Grid'] = partners['Grid'].astype(str)
growth_cw['Grid'] = growth_cw['Grid'].astype(str)
# Hago el merge
growth_cw = growth_cw.merge(partners[cols],on=['Grid'],how='left')
# Ordeno las columnas
cols = ['Grid','Id','Name','City','Area','Business','Tipo','Nombre','Owner','Fecha_Cierre','Comision']
growth_cw = growth_cw[cols]

### CARGA

In [13]:
# Carga Crudo Sales Growth
sheet_id = '1OnvLLu1PXGSTqdxXTBzlI-M0gCQLZt21POmdADJTMDY'
wks_name = 'Crudo Sales'
sheet = Spread(sheet_id, wks_name, config=cred)
sheet.df_to_sheet(growth_cw, index=False, sheet=wks_name, replace=True)